## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [4]:
import os, sys
from pathlib import Path
from ns_extension.wrapper import Splatter, SplatterConfig

import pyvista as pv
pv.start_xvfb()

# import vtkmodules.all as vtk
# render_window = vtk.vtkRenderWindow()
# render_window.SetOffScreenRendering(1)


/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/pyvista/plotting/utilities/xvfb.py:48: PyVistaDeprecationWarning: This function is deprecated and will be removed in future version of PyVista. Use vtk-osmesa instead.
  warnings.warn(


Set paths to the file for running splats

In [5]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "rats/2024-07-11/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0119.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [6]:
splatter.mesh(
    depth_name="median_depth",
)


Available runs:
[0] 2025-07-11_171420


### Plot the mesh!

We can use the splatter function ```plot_mesh``` to visualize given attributes of the mesh. The inherent attributes are RGB and Normals

In [7]:
splatter.plot_mesh(attribute="RGB")

Number of points: 277426
Number of cells: 494857
Bounds: BoundsTuple(x_min=-1.4198578596115112, x_max=0.7450000047683716, y_min=-0.125, y_max=3.0850000381469727, z_min=-1.459478735923767, z_max=1.81288480758667)


Widget(value='<iframe src="http://localhost:36567/index.html?ui=P_0x76a63444a260_0&reconnect=auto" class="pyvi…

### Using semantic queries 

The mesh contains semantic features which we can query via positive and negative prompts. The goal of this is to find points that are more similar to the positive prompts compared to the negative prompts

In [8]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"],
)

Loading model from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/config.yml


[19:20:48] Auto image downscale factor of 2                                                 ]8;id=325203;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=615430;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[19:21:17] use color only optimization with sigmoid activation                                         ]8;id=239250;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=376148;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

Plot similarity maps

In [11]:
splatter.plot_mesh(attribute=similarity, rgb=False)

Number of points: 277426
Number of cells: 494857
Bounds: BoundsTuple(x_min=-1.4198578596115112, x_max=0.7450000047683716, y_min=-0.125, y_max=3.0850000381469727, z_min=-1.459478735923767, z_max=1.81288480758667)


Widget(value='<iframe src="http://localhost:36567/index.html?ui=P_0x76a4efb16aa0_3&reconnect=auto" class="pyvi…